In [1]:
import wandb
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: miklpuerto69. Use `wandb login --relogin` to force relogin


True

In [2]:
%env WANDB_PROJECT=LLM_Science_Exam

env: WANDB_PROJECT=LLM_Science_Exam


In [30]:

import pandas as pd
from datasets import Dataset,DatasetDict
from transformers import AutoTokenizer,AutoModelForMultipleChoice, TrainingArguments, Trainer, BitsAndBytesConfig
from accelerate import Accelerator
import peft
from dataclasses import dataclass
from transformers.tokenization_utils_base import PreTrainedTokenizerBase, PaddingStrategy
from typing import Optional, Union
import torch
import datetime
import numpy as np


In [31]:
# Def Functions

options = 'ABCDE'
indices = list(range(5))

option_to_index = {option: index for option, index in zip(options, indices)}
index_to_option = {index: option for option, index in zip(options, indices)}

def preprocess(example):
    # The AutoModelForMultipleChoice class expects a set of question/answer pairs
    # so we'll copy our question 5 times before tokenizing
    first_sentence = [example['prompt']] * 5
    second_sentence = []
    for option in options:
        second_sentence.append(example[option])
    # Our tokenizer will turn our text into token IDs BERT can understand
    tokenized_example = tokenizer(first_sentence, second_sentence, truncation=True) # tokenizer call using 'text_pair' which basically just adds a separator between the two sentences
    tokenized_example['label'] = option_to_index[example['answer']]
    return tokenized_example
# Following datacollator (adapted from https://huggingface.co/docs/transformers/tasks/multiple_choice)
# will dynamically pad our questions at batch-time so we don't have to make every question the length
# of our longest question.

@dataclass
class DataCollatorForMultipleChoice:
    tokenizer: PreTrainedTokenizerBase
    padding: Union[bool, str, PaddingStrategy] = True
    max_length: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    
    def __call__(self, features):
        label_name = "label" if 'label' in features[0].keys() else 'labels'
        labels = [feature.pop(label_name) for feature in features]
        batch_size = len(features)
        num_choices = len(features[0]['input_ids'])
        flattened_features = [
            [{k: v[i] for k, v in feature.items()} for i in range(num_choices)] for feature in features
        ]
        flattened_features = sum(flattened_features, [])
        
        batch = self.tokenizer.pad(
            flattened_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors='pt',
        )
        batch = {k: v.view(batch_size, num_choices, -1) for k, v in batch.items()}
        batch['labels'] = torch.tensor(labels, dtype=torch.int64)
        return batch
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

In [60]:
# Load in Model and Tokenizer
model_path = 'microsoft/deberta-v3-base'
tokenizer = AutoTokenizer.from_pretrained(model_path)


model = AutoModelForMultipleChoice.from_pretrained(model_path,
                                                    device_map={"": current_device})#,
                                                    #torch_dtype=torch.float16)
# model=peft.prepare_model_for_int8_training(model_path)



Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/home/mowgli/miniconda3/envs/textgen2/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:470: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Some weights of DebertaV2ForMultipleChoice were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['pooler.dense.weight', 'classifier.weight', 'classifier.bias', 'pooler.dense.bias']
You shou

In [21]:
# Read in Train & Test data.

current_device = Accelerator().process_index
train_df = pd.read_csv('6000_train_examples.csv')
train_df=train_df.dropna()
train_ds = Dataset.from_pandas(train_df)
train_ds = train_ds.remove_columns(['__index_level_0__'])
# tokenized_train_ds = train_ds.map(preprocess, remove_columns=['prompt', 'A', 'B', 'C', 'D', 'E', 'answer',"__index_level_0__"])

val_df = pd.read_csv('train.csv')
val_df = val_df.dropna().drop('id',axis=1)
val_ds = Dataset.from_pandas(val_df)
tokenized_val_ds = val_ds.map(preprocess, remove_columns=['prompt', 'A', 'B', 'C', 'D', 'E', 'answer'])

datasets = DatasetDict({
    "train":train_ds,
    "validation":val_ds
})
encoded_datasets = datasets.map(preprocess, remove_columns=['prompt', 'A', 'B', 'C', 'D', 'E', 'answer'])


# test_df = pd.read_csv('test.csv')
# test_df['answer'] = 'A'
# test_ds = Dataset.from_pandas(test_df)
# tokenized_test_ds = test_ds.map(preprocess, remove_columns=['prompt', 'A', 'B', 'C', 'D', 'E', 'answer'])


Map: 100%|██████████| 200/200 [00:00<00:00, 1863.85 examples/s]


In [24]:
encoded_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'label'],
        num_rows: 5997
    })
    validation: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'label'],
        num_rows: 200
    })
})

In [25]:
# Load in existing peft
# from peft import PeftModel


# peft_model = PeftModel.from_pretrained(model, 
#                                        'finetuned/checkpoint-29500', 
#                                        is_trainable=True)

In [61]:
# topk_cpu" not implemented for 'Half
import numpy as np
def compute_metrics(eval_predictions):
    predictions, label_ids = eval_predictions
    top_k_probs, top_k_idxs = torch.topk(torch.tensor(predictions), 3, dim=1)
    accuracy = (np.array(top_k_idxs[:, 0]) == label_ids).astype(np.float32).mean().item()
    accuracy_2 = (np.array(top_k_idxs[:, 1]) == label_ids).astype(np.float32).mean().item()
    accuracy_3 = (np.array(top_k_idxs[:, 2]) == label_ids).astype(np.float32).mean().item()
    map3 = accuracy + accuracy_2/2 + accuracy_3/3
    return {"accuracy": accuracy, 'map@3' : map3}

In [67]:
# The arguments here are selected to run quickly; feel free to play with them.
checkpoint_dir = 'finetuned'
current_time=datetime.datetime.now().strftime("%H:%M:%S - %B %m %y")
training_args = TrainingArguments(
    output_dir=checkpoint_dir,
    evaluation_strategy="epoch",
#     save_strategy="steps",
#     load_best_model_at_end=True,
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=10,
    weight_decay=0.01,
    report_to='wandb',
    run_name=f'{model_path}_{current_time}'
)


In [68]:
from peft import LoraConfig, get_peft_model, TaskType
lora_config = LoraConfig(
    r=64, # Rank
    lora_alpha=64,
    target_modules=["query_proj", "value_proj"],
    lora_dropout=0.05,
    bias="none",
    # fan_in_fan_out=True,
    task_type=TaskType.SEQ_CLS
)
peft_model = get_peft_model(model, 
                            lora_config)

In [69]:
print(print_number_of_trainable_model_parameters(peft_model))

trainable model parameters: 2359296
all model parameters: 186782978
percentage of trainable model parameters: 1.26%


In [70]:
# Generally it's a bad idea to validate on your training set, but because our training set
# for this problem is so small we're going to train on all our data.
trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=encoded_datasets['train'],
    eval_dataset=encoded_datasets['validation'],
    tokenizer=tokenizer,
    data_collator=DataCollatorForMultipleChoice(tokenizer=tokenizer),
    compute_metrics=compute_metrics
)

In [73]:
torch.cuda.empty_cache() 

In [74]:
# Training should take about a minute
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Map@3
1,1.520700,1.390370,0.490000,0.632500
2,1.305200,1.280091,0.495000,0.650833
3,1.264500,1.246065,0.520000,0.670000
4,1.227200,1.211914,0.525000,0.675000
5,1.181200,1.195124,0.520000,0.679167
6,1.166600,1.175067,0.515000,0.673333
7,1.146000,1.165484,0.520000,0.681667
8,1.125300,1.166053,0.515000,0.680000
9,1.131000,1.161060,0.520000,0.680000
10,1.108700,1.161381,0.525000,0.681667


TrainOutput(global_step=7500, training_loss=1.2134420491536457, metrics={'train_runtime': 743.5908, 'train_samples_per_second': 80.649, 'train_steps_per_second': 10.086, 'total_flos': 9280743063064500.0, 'train_loss': 1.2134420491536457, 'epoch': 10.0})

In [ ]:
del model,peft_model

In [78]:
model.save_pretrained('base_models/full_debertav3/full_debertav3')

In [80]:
trainer.save_model('peft_adapters/full_debertav3/')

In [ ]:
trainer.save_pretrained('')